Merge two files into one.

Description: 	merges two files based on some key-column into one file. The lines do not 
              have to be sorted.

Original written and published by:
		* Paul I.W. de Bakker, piwdebakker@mac.com
		* 4 July 2009

Written by: 	Bram van Es; Utrecht, the Netherlands
				bramiozo@gmail.com
Suggest by:	Sander W. van der Laan; Utrecht, the Netherlands; 
              s.w.vanderlaan@gmail.com.
Version:		2.0
Update date: 	2023-04-26

Usage:		python mergeTables.py --file1 [INPUT_FILE_1] --file2 [INPUT_FILE_2] --out-file [OUTPUT_FILE] (optional: --help)

TO DO
--index [INDEX_STRING] add name of index column
--format [GZIP1/GZIP2/GZIPB/NORM] add format 
(optional: --replace) add option to replace column contents
Starting merging


Load required packages.

In [106]:
%load_ext autoreload
%autoreload 2 

# Import libraries
import os
import sys
import polars as pl
import argparse
import magic
import gzip
import time

from argparse import RawTextHelpFormatter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [107]:
# set starting time
start = time.time()

# detect file delimiter
def detect_delimiter(file_path):
    with open(file_path, "rb") as f:
        # Read first 10KB of the file to determine the delimiter
        sample = f.read(10240)
        if b" " in sample:
            return " "
        else:
            return "\t"

In [108]:
# Parse arguments
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='''
        + mergeTables 2.0 beta 1+

        This script joins `in_file1` and `in_file2` based on the `indexID` column. The index column must be
        the first column in both files. The `out_file` file will be compressed with gzip and written 
        in parquet-format when out_file ends with .parquet, otherwise a space-delimited .txt is written.
        The `replace` option adds the option to replace column contents. The `verbose` option adds the
        option to print verbose output (T/F), default: F.


        This is an example call:

        python3 mergeTables.py --in_file1 /file1.txt.gz --in_file2 /file2.txt.gz --indexID VariantID --out_file /joined.txt.gz [optional: --replace VariantID; --verbose: T/F]
        ''',
        epilog='''
        + Copyright 1979-2023. Bram van Es & Sander W. van der Laan | s.w.vanderlaan@gmail.com | https://vanderlaan.science +''', 
        formatter_class=RawTextHelpFormatter)

    # parser.add_argument('--in_file1', type=str, required=True)
    # parser.add_argument('--in_file2', type=str, required=True)
    # parser.add_argument('--indexID', type=str, required=True)
    # parser.add_argument('--replace', type=str, required=False)
    # parser.add_argument('--out_file', type=str, required=True)
    # parser.add_argument('--verbose', type=str, required=False)

    # args = parser.parse_args()

    # in_file1 = args.in_file1
    # in_file2 = args.in_file2
    # indexID = args.indexID
    # replace = args.replace
    # out_file = args.out_file
    # verbose = args.verbose

# set variables
indexID = "VariantID" # parser.parse_args().indexID
in_file1 = "/Users/slaan3/PLINK/_AE_ORIGINALS/AEGS_COMBINED_EAGLE2_1000Gp3v5HRCr11/TEST1.aegs.qc.1kgp3hrcr11.idfix.variants.rsIDs.txt" # parser.parse_args().in_file1
in_file2 = "/Users/slaan3/PLINK/_AE_ORIGINALS/AEGS_COMBINED_EAGLE2_1000Gp3v5HRCr11/TEST2.aegs.qc.1kgp3hrcr11.sumstats.extract4match.txt.gz" # parser.parse_args().in_file2
# replace = "VariantID" # parser.parse_args().replace
out_file = "/Users/slaan3/PLINK/_AE_ORIGINALS/AEGS_COMBINED_EAGLE2_1000Gp3v5HRCr11/TEST.yupiter.txt.gz" # parser.parse_args().out_file
verbose = "T" # parser.parse_args().verbose


In [109]:
print("\n+ mergeTables 2.0 beta 1 +")
print(f"\n   Starting merging the following files:")
print(f"     > [{in_file1}]")
print(f"     > [{in_file2}]")
print(f"\n   Index set to [{indexID}]")
# print(f"\n   Column to replace [{replace}]")
print(f"\n   Output will be written to [{out_file}]")

# Detect file type using magic
# https://stackoverflow.com/questions/43580/how-do-i-check-the-file-type-of-a-file-in-python

mime = magic.Magic(mime=True)
mime_type_file1 = mime.from_file(in_file1) #== 'application/gzip'

if verbose == "T":
    is_gzip1_time = time.time()
    print(f"Elapsed time: {time.strftime('%H:%M:%S', time.gmtime(is_gzip1_time - start))}")

mime_type_file2 = mime.from_file(in_file2) #== 'application/gzip'

if verbose == "T":
    is_gzip2_time = time.time()
    print(f"Elapsed time: {time.strftime('%H:%M:%S', time.gmtime(is_gzip2_time - is_gzip1_time))}")

if verbose == "T":
    print(f"\n   Detected files types:")
    print(f"     > File1 gzipped? {mime_type_file1}")
    print(f"     > File2 gzipped? {mime_type_file2}")



+ mergeTables 2.0 beta 1 +

   Starting merging the following files:
     > [/Users/slaan3/PLINK/_AE_ORIGINALS/AEGS_COMBINED_EAGLE2_1000Gp3v5HRCr11/TEST1.aegs.qc.1kgp3hrcr11.idfix.variants.rsIDs.txt]
     > [/Users/slaan3/PLINK/_AE_ORIGINALS/AEGS_COMBINED_EAGLE2_1000Gp3v5HRCr11/TEST2.aegs.qc.1kgp3hrcr11.sumstats.extract4match.txt.gz]

   Index set to [VariantID]

   Output will be written to [/Users/slaan3/PLINK/_AE_ORIGINALS/AEGS_COMBINED_EAGLE2_1000Gp3v5HRCr11/TEST.yupiter.txt.gz]
Elapsed time: 00:00:00
Elapsed time: 00:00:00

   Detected files types:
     > File1 gzipped? text/plain
     > File2 gzipped? application/gzip


In [110]:
# Detect file delimiter 
if mime_type_file1:
    with gzip.open(in_file1) as f:
        file1_delimiter = detect_delimiter(in_file1)
        file1 = pl.read_csv(in_file1, separator=file1_delimiter)
else:
    with open(in_file1) as f:
        file1_delimiter = detect_delimiter(in_file1)
        file1 = pl.read_csv(in_file1, separator=file1_delimiter)

if verbose == "T":
    file1_delimiter_t = time.time()
    print(f"Elapsed time: {time.strftime('%H:%M:%S', time.gmtime(file1_delimiter_t - is_gzip2_time))}")

if mime_type_file2:
    with gzip.open(in_file2) as f:
        file2_delimiter = detect_delimiter(in_file2)
        file2 = pl.read_csv(in_file1, separator=file2_delimiter)
else:
    with open(in_file2) as f:
        file2_delimiter = detect_delimiter(in_file2)
        file2 = pl.read_csv(in_file1, separator=file2_delimiter)

if verbose == "T":
    file2_delimiter_t = time.time()
    print(f"Elapsed time: {time.strftime('%H:%M:%S', time.gmtime(file2_delimiter_t - file1_delimiter_t))}")

if verbose == "T":
    print(f"\n   Detected the following delimiters:")
    print(f"     > File1: [{file1_delimiter}].")
    print(f"     > File2: [{file2_delimiter}].")

    print(f"\n   Detected the following heads and tails:")
    print(f"     > File1:")
    print(file1.head(3))
    print(file1.tail(3))

    print(f"     > File2:")
    print(file2.head(3))
    print(file2.tail(3))


Elapsed time: 00:00:00
Elapsed time: 00:00:00

   Detected the following delimiters:
     > File1: [ ].
     > File2: [ ].

   Detected the following heads and tails:
     > File1:
shape: (3, 8)
┌───────────┬────────────┬──────────┬────────────┬────────────┬───────────┬────────────┬───────────┐
│ VariantID ┆ chromosome ┆ position ┆ OtherAllel ┆ CodedAllel ┆ CAF       ┆ AltID      ┆ Source    │
│ ---       ┆ ---        ┆ ---      ┆ eA         ┆ eB         ┆ ---       ┆ ---        ┆ ---       │
│ str       ┆ i64        ┆ i64      ┆ ---        ┆ ---        ┆ f64       ┆ str        ┆ str       │
│           ┆            ┆          ┆ str        ┆ str        ┆           ┆            ┆           │
╞═══════════╪════════════╪══════════╪════════════╪════════════╪═══════════╪════════════╪═══════════╡
│ 1:10177   ┆ 1          ┆ 10177    ┆ A          ┆ AC         ┆ 0.019774  ┆ rs36789672 ┆ 1000Gp3v5 │
│           ┆            ┆          ┆            ┆            ┆           ┆ 4          ┆          

In [111]:
# Read files using polars
# How to get rid of 'Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance.'
# https://stackoverflow.com/questions/75690784/polars-for-python-how-to-get-rid-of-ensure-you-pass-a-path-to-the-file-instead
print(f"\n   Opening file1...")
if mime_type_file1 == "application/gzip":
    file1 = pl.read_csv(gzip.open(in_file1).read(), separator=str(file1_delimiter))
else:
    file1 = pl.read_csv(in_file1, separator=str(file1_delimiter))

if verbose == "T":
    file1_t = time.time()
    print(f"Elapsed time: {time.strftime('%H:%M:%S', time.gmtime(file1_t - file2_delimiter_t))}")

print(f"\n   Opening file2...")
if mime_type_file2 == "application/gzip":
    file2 = pl.read_csv(gzip.open(in_file2).read(), separator=str(file1_delimiter))
else:
    file2 = pl.read_csv(in_file2, separator=str(file1_delimiter))

if verbose == "T":
    file2_t = time.time()
    print(f"Elapsed time: {time.strftime('%H:%M:%S', time.gmtime(file2_t - file1_t))}")



   Opening file1...
Elapsed time: 00:00:00

   Opening file2...
Elapsed time: 00:00:00


In [112]:
new_df = file1.join(file2, on=indexID, how='inner')
if verbose == "T":
    new_df_t = time.time()
    print(f"Elapsed time: {time.strftime('%H:%M:%S', time.gmtime(new_df_t - file2_t))}")

if verbose == "T":
    print(f"\n   Detected the following heads and tails in the merge file:")
    print(new_df.head(3))
    print(new_df.tail(3))



Elapsed time: 00:00:00

   Detected the following heads and tails in the merge file:
shape: (3, 17)
┌───────────┬────────────┬──────────┬──────────────┬───┬────────┬──────────┬───────────────┬───────┐
│ VariantID ┆ chromosome ┆ position ┆ OtherAlleleA ┆ … ┆ TotalN ┆ MAF      ┆ MissingDataPr ┆ HWE_P │
│ ---       ┆ ---        ┆ ---      ┆ ---          ┆   ┆ ---    ┆ ---      ┆ oportion      ┆ ---   │
│ str       ┆ i64        ┆ i64      ┆ str          ┆   ┆ i64    ┆ f64      ┆ ---           ┆ f64   │
│           ┆            ┆          ┆              ┆   ┆        ┆          ┆ f64           ┆       │
╞═══════════╪════════════╪══════════╪══════════════╪═══╪════════╪══════════╪═══════════════╪═══════╡
│ 1:10177   ┆ 1          ┆ 10177    ┆ A            ┆ … ┆ 2124   ┆ 0.418326 ┆ -0.000038     ┆ 1.0   │
│ 1:10235   ┆ 1          ┆ 10235    ┆ T            ┆ … ┆ 2124   ┆ 0.001152 ┆ 7.0622e-7     ┆ 1.0   │
│ 1:10352   ┆ 1          ┆ 10352    ┆ T            ┆ … ┆ 2124   ┆ 0.433251 ┆ 0.000004      ┆

In [115]:
# Write output file
print(f"\n   Writing output file...")

if out_file.endswith(".parquet"):
    new_df.write_parquet(out_file, 
                         compression='gzip',
                         statistics=True)
else:
    new_df.write_csv(out_file, 
                 has_header=True, separator=' ', null_value='NA',
                 batch_size=1024)

if verbose == "T":
    new_df_write_t = time.time()
    print(f"Elapsed time: {time.strftime('%H:%M:%S', time.gmtime(new_df_write_t - new_df_t))}")

end = time.time()
print(f"Total elapsed time: {time.strftime('%H:%M:%S', time.gmtime(end - start))}")

print("\n   Wow, all done. Let's have a beer buddy!\n")
print("+ Copyright 1979-2023. Bram van Es & Sander W. van der Laan | s.w.vanderlaan@gmail.com | https://vanderlaan.science +")



   Writing output file...
Elapsed time: 00:13:10
Total elapsed time: 00:13:11

   Wow, all done. Let's have a beer buddy!

+ Copyright 1979-2023. Bram van Es & Sander W. van der Laan | s.w.vanderlaan@gmail.com | https://vanderlaan.science +
